<a href="https://colab.research.google.com/github/Mariammmmmmm/modeling_labs/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%962_%D0%BF%D0%BE_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [40]:
part0 = pd.read_parquet('part_0.parquet', engine='pyarrow')
part0 = part0.dropna()
part1 = pd.read_parquet('part_1.parquet', engine='pyarrow')
part1 = part1.dropna()
part2 = pd.read_parquet('part_2.parquet', engine='pyarrow')
part2 = part2.dropna()
part0 = part0.append(part1, ignore_index=True)
part0 = part0.append(part2, ignore_index=True)
part0.head(10)

,app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,mcc,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number
0,0,0.465425,1,4,98,4,2,3,7,3,2,1,37,2,4,19,351,34,-1,1
1,0,0.000000,1,2,98,7,1,3,7,3,2,1,49,2,4,20,351,34,0,2
2,0,0.521152,1,2,98,3,1,3,7,3,2,1,37,2,4,20,351,34,0,3
3,0,0.356078,1,1,5,2,1,3,7,3,10,1,49,7,2,0,348,34,52,4
4,0,0.000000,1,2,98,7,1,3,7,3,2,1,49,2,4,16,337,53,280,5
5,0,0.516834,1,2,98,3,1,3,7,3,2,1,37,2,4,16,337,53,0,6
6,0,0.258972,1,2,98,3,1,1,6,1,2,1,2,2,2,18,334,53,75,7
7,0,0.297589,1,2,98,3,1,1,6,1,2,1,2,2,2,19,327,52,169,8
8,0,0.356078,1,1,5,2,1,1,2,1,10,1,49,7,7,13,326,52,18,9
9,0,0.333547,1,1,5,2,1,1,2,1,10,1,49,7,7,13,326,52,0,10


In [41]:
train_target = pd.read_csv('alfabattle2_train_target.csv')
train_target.head(10)

,app_id,product,flag
0,0,3,0
1,1,1,0
2,2,1,0
3,3,1,0
4,4,1,0
5,5,1,0
6,6,1,0
7,7,0,0
8,8,2,0
9,9,0,0


In [42]:
part0 = pd.merge(part0, train_target, on='app_id', how='inner')
part0.dropna()
part0.head(10)

,app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,...,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,product,flag
0,0,0.465425,1,4,98,4,2,3,7,3,...,37,2,4,19,351,34,-1,1,3,0
1,0,0.000000,1,2,98,7,1,3,7,3,...,49,2,4,20,351,34,0,2,3,0
2,0,0.521152,1,2,98,3,1,3,7,3,...,37,2,4,20,351,34,0,3,3,0
3,0,0.356078,1,1,5,2,1,3,7,3,...,49,7,2,0,348,34,52,4,3,0
4,0,0.000000,1,2,98,7,1,3,7,3,...,49,2,4,16,337,53,280,5,3,0
5,0,0.516834,1,2,98,3,1,3,7,3,...,37,2,4,16,337,53,0,6,3,0
6,0,0.258972,1,2,98,3,1,1,6,1,...,2,2,2,18,334,53,75,7,3,0
7,0,0.297589,1,2,98,3,1,1,6,1,...,2,2,2,19,327,52,169,8,3,0
8,0,0.356078,1,1,5,2,1,1,2,1,...,49,7,7,13,326,52,18,9,3,0
9,0,0.333547,1,1,5,2,1,1,2,1,...,49,7,7,13,326,52,0,10,3,0


In [43]:
X = part0.drop('flag', axis='columns')
y = part0.flag
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=42)
X_train.head(10)

,app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,...,country,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,product
5201636,22774,0.294135,1,1,107,2,1,1,2,1,...,1,82,13,1,14,54,51,24,1154,1
8573,37,0.449287,1,2,98,3,1,1,6,1,...,1,2,2,7,20,322,46,0,52,0
587296,2695,0.408972,1,1,5,2,1,1,2,1,...,1,19,1,7,17,175,40,1,161,1
4997574,21926,0.512351,4,1,175,2,1,1,2,1,...,3,2,6,7,20,45,24,1,443,0
4131718,18332,0.354176,1,1,175,1,1,1,1,1,...,1,1,14,3,16,76,30,0,3404,1
579935,2663,0.336345,1,1,75,2,1,1,2,1,...,1,1,11,2,20,316,20,110,7,0
1653353,7354,0.476811,1,2,27,3,1,1,4,1,...,1,3,2,3,9,59,45,2,441,1
177819,826,0.426544,1,2,6,7,1,1,1,1,...,1,4,2,2,15,267,43,24,48,0
5392893,23579,0.348838,1,1,54,2,1,1,2,1,...,1,7,1,5,19,281,2,0,160,2
466590,2156,0.256357,1,1,6,1,1,1,1,1,...,1,57,4,4,10,39,31,17,179,0


In [7]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from catboost import CatBoostClassifier, metrics

model = CatBoostClassifier()

parameters = { 'depth' : sp_randInt(4, 10),
               'learning_rate': sp_randFloat(),
              'iterations': sp_randInt(10, 100)
}

randn = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                           cv = 2, n_iter = 10, n_jobs=-1)
randn.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0:	learn: 0.1094727	total: 1.16s	remaining: 1m 13s
1:	learn: 0.1050530	total: 2.32s	remaining: 1m 11s
2:	learn: 0.1029739	total: 3.31s	remaining: 1m 7s
3:	learn: 0.1016459	total: 4.46s	remaining: 1m 6s
4:	learn: 0.0984860	total: 5.58s	remaining: 1m 5s
5:	learn: 0.0969980	total: 6.58s	remaining: 1m 3s
6:	learn: 0.0959844	total: 7.54s	remaining: 1m 1s
7:	learn: 0.0938044	total: 8.67s	remaining: 1m
8:	learn: 0.0920394	total: 9.77s	remaining: 59.7s
9:	learn: 0.0906365	total: 10.8s	remaining: 58.4s
10:	learn: 0.0899148	total: 11.9s	remaining: 57.2s
11:	learn: 0.0881480	total: 13s	remaining: 56.5s
12:	learn: 0.0864723	total: 14.1s	remaining: 55.4s
13:	learn: 0.0852424	total: 15.3s	remaining: 54.5s
14:	learn: 0.0842258	total: 16.4s	remaining: 53.4s
15:	learn: 0.0834110	total: 17.5s	remaining: 52.4s
16:	learn: 0.0824858	total: 18.5s	remaining: 51.3s
17:	learn: 0.0815475	total: 19.7s	remaining: 50.3s
18:	learn: 0.0811706	total: 20.6s	remaining: 48.9s
19:	learn: 0.0798994	total: 21.8s	remaining:

RandomizedSearchCV(cv=2,
                   estimator=<catboost.core.CatBoostClassifier object at 0x7f3c6247c6d0>,
                   n_jobs=-1,
                   param_distributions={'depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f3c6247cb50>,
                                        'iterations': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f3c6247c850>,
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f3c6247ca10>})

In [45]:
print("Results from Random search " )
print("\n The best estimator across ALL searched params:\n" ,randn.best_estimator_)
print("\n The best score across ALL searched params:\n",randn.best_score_)
print("\n The best parameters across ALL searched params:\n", randn.best_params_)

Results from Random search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 0.9859109209280235

 The best parameters across ALL searched params:
 {'depth': 9, 'iterations': 64, 'learning_rate': 0.9993486625980919}


In [59]:
from sklearn.metrics import accuracy_score, roc_auc_score
print('the test accuracy is :{:.6f}'.format(accuracy_score(y_test,randn.best_estimator_.predict(X_test))))
val_roc_auc_score = roc_auc_score(y_test, randn.best_estimator_.predict(X_test))
print('the test roc_auc accuracy is ',val_roc_auc_score)

the test accuracy is :0.805926
the test roc_auc accuracy is  0.7245451042136632
